In [22]:
import os, sys
import numpy as np
import pandas as pd
import random
from scipy import stats as st
import itertools
import operator
import heapq as hq
import torch

from tqdm.notebook import trange
from tqdm import tqdm

# Init steps

In [39]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks : helpers.ipynb contains a set of tools and functions used in this notebook
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

In [24]:
for p in ['../spotlight_ext']:
    module_path = os.path.abspath(os.path.join(base_dir, p))
    if module_path not in sys.path:
        sys.path.append(module_path)

random_state = np.random.RandomState(2020)

# Prepare models/datasets

In [25]:
# Load the pretrained models "lstm" (entire_model_1m_20interactions.pt) and "pooling" (pooling_model_1m_20interactions.pt) presents in the models folder

# implicit_model = load_model('implicit_factorization')
lstm_model = load_model(model_type='entire') # the code to create this model is in misc.ipynb (see section "train model")
pooling_model = load_model('pooling') # the code to create this model is in brute_force_rec_expl.ipynb (see section "train and save pooling model")

pretrained_models = {
    'lstm': lstm_model,
    'pooling': pooling_model,
}

In [26]:
# Get the dataset Movielens with the variant 1M. Then divide it into a training set and a testing set. It also limits the length of each sequence of elements in the 2 sets to 20.

from spotlight.cross_validation import random_train_test_split
from spotlight.datasets.movielens import get_movielens_dataset

# get dataset, more information here : https://grouplens.org/datasets/movielens/
dataset = get_movielens_dataset(variant='1M')
train, test = random_train_test_split(dataset, random_state=random_state)

max_sequence_length = 20 # Maximum sequence length. Subsequences shorter than this will be left-padded with zeros.
train = train.to_sequence(max_sequence_length=max_sequence_length)
test = test.to_sequence(max_sequence_length=max_sequence_length) # test is a SequenceInteractions object, here is the documentation on this object : https://maciejkula.github.io/spotlight/interactions.html

In [40]:
# Compute cosine similarity matrix for item embeddings using GPU
pooling_sims_matrix = gpu_embeddings_to_cosine_similarity_matrix(
    pooling_model._net.item_embeddings(
        torch.arange(0, dataset.num_items, dtype=torch.int64)
    )).detach().numpy()

# Compute item-item similarity matrix using Jaccard similarity
jaccard_sims_matrix = compute_sim_matrix(dataset, 'jaccard')

  0%|          | 0/6041 [00:00<?, ?it/s]

[]
[5. 3. 3. 4. 5. 3. 5. 5. 4. 4. 5. 4. 4. 4. 5. 4. 3. 4. 5. 4. 3. 3. 5. 5.
 3. 5. 4. 4. 4. 3. 4. 4. 4. 4. 4. 4. 5. 5. 4. 5. 5. 5. 4. 4. 4. 5. 5. 4.
 5. 4. 4. 4. 4.]
[5. 4. 4. 3. 4. 4. 5. 3. 3. 3. 5. 4. 3. 3. 2. 5. 3. 4. 3. 4. 2. 3. 2. 3.
 4. 3. 5. 3. 5. 4. 2. 4. 5. 5. 4. 5. 4. 4. 2. 3. 5. 3. 3. 4. 5. 4. 3. 3.
 3. 5. 4. 4. 4. 5. 4. 5. 3. 3. 4. 3. 3. 2. 5. 4. 5. 3. 5. 5. 3. 3. 4. 4.
 2. 5. 5. 3. 4. 4. 5. 3. 2. 3. 4. 3. 4. 3. 4. 3. 3. 5. 4. 3. 5. 5. 4. 1.
 3. 3. 2. 5. 5. 4. 4. 4. 5. 3. 4. 4. 3. 4. 4. 3. 3. 3. 5. 5. 5. 4. 3. 5.
 4. 2. 3. 3. 5. 2. 5. 1. 5.]
[4. 2. 3. 4. 3. 4. 4. 4. 3. 3. 5. 3. 5. 4. 5. 4. 5. 3. 2. 5. 4. 3. 4. 1.
 4. 4. 2. 5. 5. 4. 4. 5. 5. 5. 5. 3. 4. 5. 5. 3. 4. 4. 2. 4. 5. 4. 4. 4.
 4. 5. 4.]
[5. 3. 4. 4. 4. 5. 5. 4. 2. 5. 5. 4. 4. 4. 4. 5. 1. 5. 5. 5. 5.]
[4. 4. 5. 3. 2. 5. 4. 4. 2. 3. 2. 3. 4. 3. 1. 4. 3. 4. 1. 2. 4. 5. 2. 4.
 4. 4. 1. 3. 5. 3. 4. 4. 3. 3. 4. 3. 2. 4. 3. 2. 3. 2. 3. 2. 3. 4. 5. 4.
 2. 5. 2. 2. 5. 1. 3. 4. 3. 2. 5. 5. 1. 3. 5. 3. 2. 3. 3. 4. 3. 3. 3. 1.

In [33]:
print(jaccard_sims_matrix[0])
print(jaccard_sims_matrix[1])
print(jaccard_sims_matrix[0].shape)
print(jaccard_sims_matrix[1].shape)

[[1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.03482099e-01 ... 0.00000000e+00
  5.79710145e-04 0.00000000e+00]
 [0.00000000e+00 1.03482099e-01 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 5.79710145e-04 0.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]]
[[False False False ... False False False]
 [False  True  True ... False False False]
 [False  True False ... False False False]
 ...
 [False False False ... False False False]
 [False False  True ... False False False]
 [False  True False ... False False False]]
(3707, 3707)
(6041, 3707)


In [38]:
print(max(dataset.user_ids))
print(min(dataset.user_ids))
print(dataset.num_users)
print(max(dataset.item_ids))
print(min(dataset.item_ids))
print(dataset.num_items)

6040
1
6041
3706
1
3707


# Various implemented Strategies

In [41]:
class BaseStrategy:
    class_name = None

    def __init__(self, item, interactions, max_length, init_budget,  model=None, random_pick=False):

        self.target_item = item
        self.original_interactions = interactions
        self.max_length = max_length
        self.visited_ = set()
        self.model = model
        self.last_comb_cost = 0
        self.random_pick = random_pick
        self.top_k = 10
        self.budget = init_budget

    # Must be implemented by subclasses. Used to select the next item to recommand to the user.
    def next_comb(self, reverse=False):
        raise NotImplementedError

    
    # This method takes a "number" argument and returns a list of 0-bit positions in the binary representation of "number".
    def _get_pos(self, number):
        bits = []
        for i, c in enumerate(bin(number)[:1:-1], 1):
            if c == '0':
                bits.append(i)
        return bits

    # Method to reset the costs of the last recommended combination
    def reset_costs(self):
        self.last_comb_cost = 0

    # Returns the initial budget
    def get_init_budget(self):
        return self.budget

### RandomSelection
#### This class is a subclass of the "BaseStrategy" class, representing a random item selection strategy for the sequential recommendation task.

In [42]:
class RandomSelection(BaseStrategy):
    class_name = 'Random'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget)

    # The _next_item method selects a random integer between 1 and 2 raised to the power of the maximum length of the sequence. 
    # It checks if the integer is already in the set of visited integers, and if it is, selects another integer until a non-visited integer is found. 
    # Finally, the selected integer is added to the set of visited integers, and returned.
    def _next_item(self):
        self.budget -= 1
        
        number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        while number in self.visited_:
            number = random.sample(range(1, pow(2, self.max_length)), 1)[0]
        self.visited_.add(number)
        
        return number
    
    # The next_comb method generates a new sequence by removing items at positions indicated by the binary digits of the integer returned 
    # by _next_item from the original sequence of interactions. 
    # The resulting sequence, along with the current budget, is returned as a tuple.
    def next_comb(self, reverse=False):
        number = self._next_item()

        bits = self._get_pos(number)
        seq = np.delete(self.original_interactions, bits)

        return (seq, self.budget)

### LossSimilarSelection
#### This class inherits from the BaseStrategy class. This class defines a search strategy for selecting items based on their similarity to previously selected items, while also considering their loss (difference between predicted and actual values) in a ranking problem.

In [43]:
class LossSimilarSelection(BaseStrategy):
    class_name = 'BFS'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, early_term=False):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        mask = [False] * len(self.original_interactions)
        t_score = StaticVars.INT_MAX
        is_solved = 0

        self.q = Queue()
        self.q.enqueue((mask, t_score, is_solved))

        self.thres = len(self.original_interactions) + 1
        self.early_termination = early_term

    # Helper : This method is called whenever a solution is found for the current 
    # mask of the items. It computes the loss of the solution and updates the queue 
    # accordingly.
    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    # Helper : 
    def _next_item(self):
        mask, t_score, is_solved = self.q.dequeue()
        while self.early_termination and sum(mask) == self.thres:
            q_data = self.q.dequeue()
            if q_data is None: 
                break
            mask, t_score, is_solved = q_data

        if is_solved == 2:
            t_score, kth_score = self.get_score(mask)

            if (t_score / kth_score) < 1: 
                self.thres = sum(mask)

        return (is_solved, mask, self.budget)

    # The next_comb method returns the next combination of items and the remaining budget.
    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q.size() > 0:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy()) # If there is a True in the mask, the value at the corresponding index is masked
            self._update_queue(solved_flag)
        else: 
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    # This method computes the loss of the solution. If the solution is not yet solved, it searches for the next combination of items 
    # to evaluate by calling the search method. 
    # If the solution is solved, it searches for the previous combination of items by calling the search method with forward=False.
    def compute_loss(self, is_solved=False):
        self.last_comb_cost = 0

        if not is_solved: 
            self.search(forward=True, s=is_solved)
        else: 
            self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        """
        Searches for the next combination of items based on the search direction.

        Args:
            forward (bool, optional): If True, the search is performed forward, otherwise backward.
            s (bool, optional): Indicates whether a solution is found.
        """
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0] #if forward is True it means we keep index of items where the m_mask is not True
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_score(self, d):
        """
        Computes the score of an item based on its predicted value and rank among the top-k items.

        Args:
            d (array_like): Boolean mask indicating the selected items.

        Returns:
            Tuple: A tuple containing the score of the target item and its rank among the top-k items.
        """
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

        self.budget -= 1
        # predict next top-k items about to be selected
        preds = self.model.predict(perm)
        preds[perm] = -StaticVars.FLOAT_MAX
        rk_data = st.rankdata(-preds, method='ordinal')

        return (preds[self.target_item], preds[(rk_data == self.top_k).nonzero()][0])

    def add(self, d, s):
        """
        Adds a new combination to the queue and updates the visited set.

        Args:
            d (array_like): Boolean mask representing the combination of items.
            s (bool): Indicates whether a solution is found.
        """
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))

            if not s:
                t_score, kth_score = self.get_score(d)

                if self.q.size() == 0: self.q.enqueue((d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))

                if t_score < self.q.get(0)[1]:  # get only the assigned score
                    self.q.setter(0, (d.copy(), t_score, 1 if (t_score / kth_score) < 1 else 0))
            else:
                self.q.enqueue((d.copy(), StaticVars.INT_MAX, 2))

            self.visited_.add(mask_to_int)

### BiDirectionalSelection

In [45]:
class BiDirectionalSelection(BaseStrategy):
    class_name = 'BiDirectional'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.tiebraker = itertools.count()
        self.q = [(1, StaticVars.INT_MAX, next(self.tiebraker), [False] * len(self.original_interactions), self.budget)]
        hq.heapify(self.q)

        self.alpha = alpha
        self.norm = normalization

    def _update_queue(self, is_solved):
        self.compute_loss(is_solved)

    def _next_item(self):
        is_solved, _, _, mask, budget = hq.heappop(self.q)
        return (is_solved, mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget
        if self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def compute_loss(self, is_solved=False):
        self.search(forward=True, s=is_solved)
        self.search(forward=False, s=is_solved)

    def search(self, forward=True, s=False):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask) if forward else m_mask)[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask, s)
                m_mask[idx] = not m_mask[idx]

    def get_custom_score(self, c):
        return c / self.max_length

    def get_score(self, d):
        self.budget -= 1

        # predict next top-k items about to be selected
        perm = np.ma.compressed(np.ma.masked_array(self.original_interactions, mask=d))
        preds = self.model.predict(perm)

        if self.norm == 'kth_norm':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            t_score = preds[self.target_item] / preds[(rk_data == self.top_k).nonzero()][0]
        elif self.norm == 'rescale':
            preds[perm] = -StaticVars.FLOAT_MAX
            rk_data = st.rankdata(-preds, method='ordinal')

            max_val = rk_data[0]
            min_val = rk_data[-1]
            t_score = (max_val - preds[self.target_item]) / (max_val - min_val)
        else:  # default case
            tensor = F.softmax(torch.from_numpy(preds).float(), dim=0)
            preds = tensor.numpy()
            preds[perm] = -StaticVars.FLOAT_MAX

            t_score = preds[self.target_item]

        return self.alpha * t_score + (1 - self.alpha) * self.get_custom_score(np.sum(d))

    def add(self, d, s):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            t_score = self.get_score(d)
            hq.heappush(self.q, (int(not s), t_score, next(self.tiebraker), d.copy(), self.budget))

            self.visited_.add(mask_to_int)

# BruteForceSelection

In [46]:
class BruteForceSelection(BaseStrategy):
    class_name = 'BruteForce'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model):
        super().__init__(item, interactions, max_sequence_length, init_budget, model)

        self.q = Queue()
        self.q.enqueue(([False] * len(self.original_interactions), self.budget))

    def _expand_queue(self):
        m_mask = np.ma.getmask(self.ma_arr).copy()
        valid_items = np.where(np.logical_not(m_mask))[0]
        if valid_items.size > 1:
            for idx in valid_items:
                m_mask[idx] = not m_mask[idx]
                self.add(m_mask)
                m_mask[idx] = not m_mask[idx]

    def _next_item(self):
        mask, budget = self.q.dequeue()
        return (mask, budget)

    def next_comb(self, reverse=False):
        budget = self.budget

        if reverse: self.q.clear()

        if self.q.size() > 0:
            item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())
            self._expand_queue()
        else:
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

    def add(self, d):
        mask_to_int = int(''.join(map(str, d.astype(int))), 2)
        if (mask_to_int not in self.visited_) and (self.budget > 0):
            self.budget -= 1
            self.q.enqueue((d.copy(), self.budget))
            self.visited_.add(mask_to_int)

# ComboSelection

In [47]:
class ComboSelection(BiDirectionalSelection):
    class_name = 'Combo'

    def __init__(self, item, interactions, max_sequence_length, init_budget, model, weights=(1, 0), alpha=0.9, normalization='default'):
        super().__init__(item, interactions, max_sequence_length, init_budget, model, weights, alpha, normalization)

        self.alpha = 1

        self.q_init = Queue()
        self.q_init.enqueue((StaticVars.INT_MAX, [False] * len(self.original_interactions), self.budget))
        self.init_queue()

        self.tiebraker = itertools.count()
        self.q = []
        hq.heapify(self.q)

        self.alpha = alpha

    def init_queue(self):
        _, m_mask, budget = self.q_init.dequeue()
        m_mask = np.asarray(m_mask)

        valid_items = np.where(np.logical_not(m_mask))[0]
        for idx in valid_items:
            m_mask[idx] = not m_mask[idx]

            mask_to_int = int(''.join(map(str, m_mask.astype(int))), 2)
            if (mask_to_int not in self.visited_) and (self.budget > 0):
                t_score = self.get_score(m_mask)
                self.q_init.enqueue((t_score, m_mask.copy(), self.budget))

                self.visited_.add(mask_to_int)

            m_mask[idx] = not m_mask[idx]

        pair_combs = []
        for c in itertools.combinations(range(len(self.original_interactions)), 2):
            m = [False] * len(self.original_interactions)
            m[c[0]], m[c[1]] = not m[c[0]], not m[c[1]]
            pair_combs.append((self.q_init.get(c[0])[0] + self.q_init.get(c[1])[0], m.copy()))

        pair_combs.sort(key=operator.itemgetter(0))
        for c in pair_combs:
            self.budget -= 1
            self.q_init.enqueue((0, c[1], self.budget))

    def next_comb(self, reverse=False):
        budget = self.budget

        if self.q_init.size() > 0:
            s, item_mask, budget = self.q_init.dequeue()
            item_mask = np.asarray(item_mask)
            solved_flag = False
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self.add(item_mask, False)
        elif self.q:
            solved_flag, item_mask, budget = self._next_item()
            self.ma_arr = np.ma.masked_array(self.original_interactions, mask=item_mask.copy())

            self._update_queue(solved_flag)
        else: self.ma_arr = np.ma.masked_array(self.original_interactions, mask=True)

        seq = np.ma.compressed(self.ma_arr)
        return (seq, budget) if len(seq) else (None, budget)

### Temporary code

In [ ]:
class MostSimilarSelection(BaseStrategy):
    class_name = 'Sim-Matrix'

    supported_sim_matrix = {
        'pooling': pooling_sims_matrix,
        'jaccard': jaccard_sims_matrix
    }

    def __init__(self, item, interactions, max_sequence_length, model, sim_type='pooling'):
        super().__init__(item, interactions, max_sequence_length)

        self.visited_.add(0)
        self.reverse_checks = []
        self.is_materialized = False

        self._get_sim_ranking(sim_type)

    def next_comb(self, reverse=False):
        if reverse:
            self._materialize_list()
            selected_item_indices = self.reverse_checks.pop(
                random.randrange(len(self.reverse_checks)) if self.random_pick else 0
            ) if len(self.reverse_checks) else []
        else:
            self.visited_.add(max(self.visited_) + 1)
            selected_item_indices = np.where(np.isin(
                self.rk_items,
                list(set(self.rk_items).difference(set(self.visited_)))
            ))[0]
        seq = self.original_interactions[selected_item_indices] if len(selected_item_indices) else None
        return seq

    def _get_sim_ranking(self, sim_type):
        ranked_items = st.rankdata(self.supported_sim_matrix[sim_type][self.target_item, self.original_interactions])
        self.rk_items = self.max_length - ranked_items + 1

    def _materialize_list(self):
        if not self.is_materialized:
            psize = len(self.visited_) - 1  # do not consider initial added zero value
            # do not take account none/all excluded interacted items
            prods = sorted(list(map(list, itertools.product([0, 1], repeat=psize)))[1:-1], key=sum)
#             last_item_indices = np.where(np.isin(
#                 self.rk_items,
#                 list(set(self.rk_items).difference(set(self.visited_)))
#             ))

            lvisited_ = np.asarray(list(self.visited_))[1:]
            for p in prods:
                self.reverse_checks.append(np.where(np.isin(
                    self.rk_items,
                    list(set(self.rk_items).difference(lvisited_[np.nonzero(np.multiply(p, lvisited_))])))
                ))

            self.is_materialized = True

In [ ]:
class MostSimilarSelectionByJaccard(MostSimilarSelection):
    class_name = 'Jaccard-on-Sim-Matrix'

    def __init__(self, item, interactions, max_sequence_length, model):
        super().__init__(item, interactions, max_sequence_length, model, 'jaccard')

In [ ]:
# cosine_on_embeddings_cfs = [
#     _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim', init_budget=1000)
# ]
# jaccard_on_embeddings_cfs = [
#     _find_cfs(test, pretrained_models['lstm'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
#     _find_cfs(test, pretrained_models['pooling'], [3, 5, 7], no_users=500, backend='most_sim_jaccard', init_budget=1000),
# ]

# %store cosine_on_embeddings_cfs
# %store jaccard_on_embeddings_cfs

# Get backend strategy

In [48]:
def get_backend_strategy(backend):
    if 'random' == backend:
        return RandomSelection
    elif 'most_sim' == backend:
        return MostSimilarSelection
    elif 'most_sim_jaccard' == backend:
        return MostSimilarSelectionByJaccard
    elif 'bfs' == backend:
        return LossSimilarSelection
    elif 'random_most_sim' == backend:
        return RandomMostSimilarSelection
    elif 'random_loss_sim' == backend:
        return RandomLossSimilarSelection
    elif 'fixed_loss_sim' == backend:
        return FixedRankingLossSimilarSelection
    elif 'dfs_loss_sim' == backend:
        return DFSwithLossSelection
    elif 'dfs_fixed_loss_sim' == backend:
        return DFSwithFixedRankingLossSelection
    elif 'bestFS_loss' == backend:
        return BestFSLossSelection
    elif 'bestFS_fixed_loss' == backend:
        return BestFSFixedLossSelection
    elif 'topdown_loss' == backend:
        return TopDownBestFSLossSelection
    elif 'bidirectional' == backend:
        return BiDirectionalSelection
    elif 'brute_force' == backend:
        return BruteForceSelection
    elif 'combo' == backend:
        return ComboSelection
    else: print('Unknown strategy')

# Run implemented strategies

In [55]:
# get currently working directory
base_dir = os.getcwd()

# load functions from other notebooks
helpers_file = os.path.join(base_dir, 'helpers.ipynb').replace("\\", "/")
%run $helpers_file

backend = 'random'
random_cfs = [

    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000)
    # _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000)
]

%store random_cfs


target position loop:   0%|          | 0/4 [00:00<?, ?it/s]

The backend used is: Random


users loop:   0%|          | 0/10 [00:00<?, ?it/s]

top [   0 3474 3473 3472 3467 3463 3462 3460 3458 3454 3451 3449 3448 3444
 3441 3435 3429 3427 3476 3477]
find_best_jaccard top items : [1, 3495, 3494, 3493, 3488, 3484, 3483, 3481, 3479, 3475, 3472, 3470, 3469, 3465, 3462, 3456, 3450, 3448, 3497, 3498]
[230 257 359 130 358 329 372 227 324 301 253 239 107 305 266 315  60 331
 123 357]
20
<class 'numpy.ndarray'>
[1, 3495, 3494, 3493, 3488, 3484, 3483, 3481, 3479, 3475, 3472, 3470, 3469, 3465, 3462, 3456, 3450, 3448, 3497, 3498]
20
top [   0 3445 3444 3443 3441 3431 3427 3425 3416 3413 3448 3399 3393 3390
 3388 3379 3378 3377 3372 3365]
find_best_jaccard top items : [1, 3466, 3465, 3464, 3462, 3452, 3448, 3446, 3437, 3434, 3469, 3420, 3414, 3411, 3409, 3400, 3399, 3398, 3393, 3386]
[384 461 236 478  86 457 511  60 323 443 360 473 472 514 456 481 119 450
 453 444]
20
<class 'numpy.ndarray'>
[1, 3466, 3465, 3464, 3462, 3452, 3448, 3446, 3437, 3434, 3469, 3420, 3414, 3411, 3409, 3400, 3399, 3398, 3393, 3386]
20
top [   0 3481 3482 3483 348

target position loop: 20it [00:10,  1.89it/s]              

...

target position loop: 40it [00:30,  1.24it/s]

users loop:   0%|          | 0/10 [00:00<?, ?it/s]

top [   0 3462 3456 3455 3451 3449 3448 3445 3444 3441 3434 3431 3427 3416
 3401 3399 3396 3394 3463 3467]
find_best_jaccard top items : [1, 3483, 3477, 3476, 3472, 3470, 3469, 3466, 3465, 3462, 3455, 3452, 3448, 3437, 3422, 3420, 3417, 3415, 3484, 3488]
[230 257 359 130 358 329 372 227 324 301 253 239 107 305 266 315  60 331
 123 357]
20
<class 'numpy.ndarray'>
[1, 3483, 3477, 3476, 3472, 3470, 3469, 3466, 3465, 3462, 3455, 3452, 3448, 3437, 3422, 3420, 3417, 3415, 3484, 3488]
20
top [   0 3372 3369 3368 3365 3360 3356 3350 3337 3334 3330 3328 3324 3320
 3317 3375 3315 3302 3299 3289]
find_best_jaccard top items : [1, 3393, 3390, 3389, 3386, 3381, 3377, 3371, 3358, 3355, 3351, 3349, 3345, 3341, 3338, 3396, 3336, 3323, 3320, 3310]
[384 461 236 478  86 457 511  60 323 443 360 473 472 514 456 481 119 450
 453 444]
20
<class 'numpy.ndarray'>
[1, 3393, 3390, 3389, 3386, 3381, 3377, 3371, 3358, 3355, 3351, 3349, 3345, 3341, 3338, 3396, 3336, 3323, 3320, 3310]
20
top [   0 3507 3503 3499 349

target position loop: 40it [00:41,  1.03s/it]

Stored 'random_cfs' (list)


In [ ]:
backend = 'bfs'

bfs_yloss_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users= 10, init_budget=1000, early_term=True),
]

# %store bfs_yloss_cfs_positive
%store bfs_yloss_cfs

In [ ]:
backend='bidirectional'

bidirectional_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.999, normalization='default'),
]

%store bidirectional_cfs

In [ ]:
backend='brute_force'

brute_force_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=100000),
]

%store brute_force_cfs

In [ ]:
backend='combo'
combo_cfs = [
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=1e-3, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.5, normalization='default'),
    _find_cfs(test, pretrained_models['lstm'], get_backend_strategy(backend), [11, 13, 15, 17], missing_target_in_topk=True, sim_matrix=jaccard_sims_matrix, no_users=10, init_budget=1000, alpha=0.999, normalization='default'),
]

%store combo_cfs